## training code (from Colab)

In [2]:
# ------------------------------
# STEP 0: Unzip the dataset
# ------------------------------
import zipfile
import os

zip_path = "/content/dogs-vs-cats.zip"
extract_path = "/content/dogs-vs-cats"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check the extracted folders
os.listdir(extract_path)

# ------------------------------
# STEP 1: Import required libraries
# ------------------------------
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import pandas as pd

# ------------------------------
# STEP 2: Set dataset paths
# ------------------------------
train_dir = os.path.join(extract_path, "train")
test_dir = os.path.join(extract_path, "test")

# ------------------------------
# STEP 3: Data preprocessing
# ------------------------------
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Only rescaling for test data
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Create train data loader
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Create test data loader
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# ------------------------------
# STEP 4: Build CNN model
# ------------------------------
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# ------------------------------
# STEP 5: Compile the model
# ------------------------------
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ------------------------------
# STEP 6: Train the model
# ------------------------------
history = model.fit(
    train_data,
    epochs=5,                # You can increase for better accuracy
    validation_data=test_data
)

# ------------------------------
# STEP 7: Evaluate the model
# ------------------------------
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy*100:.2f}%")

# ------------------------------
# STEP 8: Save the model
# ------------------------------
model.save("dogs_vs_cats_model.h5")
print("Model saved as dogs_vs_cats_model.h5")
